# What's new in Brightway 2.5

## Backwards compatibility

Compatilibility with Brightway 2 has been maintained whenever possible, but there are a few cases where compatiblity could not be kept.

* In `bw2calc`, the `LCA` class now takes over responsibility for all types of LCA calculations, including Monte Carlo.
* In `bw2data`, `Database.get()` and `Database().get()` are no longer supported. Use `get_node(database="something", **other_filters)` instead.

In [ ]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi

In [ ]:
# if "2.5 examples in action" in bd.projects:
#     bd.projects.delete_project("2.5 examples in action", True)
bd.projects.set_current("2.5 examples in action")

In [ ]:
bi.bw2setup()

In [ ]:
if 'Mobility example' in bd.databases:
    bd.Database("Mobility example").delete_instance()
    from bw2data.parameters import ProjectParameter, DatabaseParameter, ActivityParameter, ParameterizedExchange, Group
    ProjectParameter.delete().execute()
    DatabaseParameter.delete().execute()
    ActivityParameter.delete().execute()
    ParameterizedExchange.delete().execute()
    Group.delete().execute()
    
bi.add_example_database()

In [ ]:
if 'US EEIO 1.1' not in bd.databases:
    bi.useeio11()

In [ ]:
product = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product')
activity = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'process')
emission = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'emission')

In [ ]:
mobility = bd.Database('Mobility example')

In [ ]:
steel = bd.get_node(name='Steel')
steel['properties'] = {'carbon content': {'amount': 0.01}}
steel['classifications'] = {'ISIC': {'code': '2410', 'system': 'ISIC Rev. 4'}}
steel.save()

In [ ]:
ipcc = bd.Method(('IPCC', 'uncertain'))
ipcc.register()
ipcc.write([(('Mobility example', 'CO2'), {'uncertainty_type': 3, 'amount': 1, 'loc': 1, 'scale': 0.1})])

In [ ]:
if 'EXIOBASE 3.8.1 2017 monetary' not in bd.databases:
    bi.exiobase_monetary()

In [ ]:
exiobase = bd.Database("EXIOBASE 3.8.1 2017 monetary")

# `bw2data`

`Make steel` -> `Activity`

`Drive motorcycle` -> `Activity`

`Grow sugarbeet` -> `Activity`

`CO2` -> `Activity`????

## Change in preferred nomenclature

`Activity` -> `Node`

`Exchange` -> `Edge`

In [ ]:
isinstance(steel, bd.Node)

But also:

`steel.edges()`, `steel.new_edge()`, `mobility.new_node()`

## OMGWTFBBQ

In [ ]:
[act 
 for act in bd.Database('Mobility example')
 if act['name'] == 'Steel'
][0]

In [ ]:
bd.get_node(database='Mobility example', code='Steel')

## Any attribute

In [ ]:
steel = bd.get_node(name='Steel')
steel['foo'] = 'bar'
steel.save()

In [ ]:
bd.get_node(foo='bar')

In [ ]:
bd.get_node(database='Mobility example', foo='bar')

## `get_node` returns only one node

`bw2data.errors.UnknownObject` if no nodes can be found.

`bw2data.errors.MultipleResults` if more than one node.

## `get_id` and the removal of `mapping`

No more `mapping.pickle` file. `id` comes from the database and is preferred.

In [ ]:
steel.id

In [ ]:
bd.get_id(steel)

In [ ]:
bd.mapping[steel.key]

## Let's write SQL

`Node` table has columns for `code`, `database`, `location`, `name`, `product`, `type`.

In [ ]:
from bw2data.backends import ActivityDataset as AD
AD.update(name="Wow, this is some car!").where(AD.name == 'Combustion car').execute()

In [ ]:
bd.get_node(name="Wow, this is some car!")

## `Activity` attribute lookups

Make the world more magical. Boo:

In [ ]:
[value for key, value in steel['properties'].items() if key == 'carbon content'][0]

Jawohl!

In [ ]:
steel['carbon content']

In [ ]:
steel['ISIC']

## Reference products

No!!

In [ ]:
[exc for exc in steel.exchanges() if exc['type'] == 'production'][0]

Yes!!

In [ ]:
steel.rp_exchange()

Double yes!!

In [ ]:
exc = steel.rp_exchange()
exc['properties'] = {'iron content': 0.98}
exc.save()

In [ ]:
steel['iron content']

## Filepaths are instances of `pathlib.Path`

`Path` objects are [pretty great](https://treyhunner.com/2018/12/why-you-should-be-using-pathlib/), you should [use them](https://docs.python.org/3/library/pathlib.html).

In [ ]:
type(bd.projects.dir), type(bd.projects.logs_dir)

In [ ]:
mobility.dirpath_processed() / "filename.zip"

## Easier access to `Datapackages`

This guy is dumb!

In [ ]:
from fs.zipfs import ZipFS
import bw_processing as bwp

bwp.load_datapackage(ZipFS(mobility.filepath_processed()))

That's better...

In [ ]:
mobility.datapackage()

## IOTable improvements

Not yet released - not in conference notebook  `¯\_(ツ)_/¯`

In [ ]:
act = exiobase.random()
act

In [ ]:
for exc, _ in zip(act.technosphere(), range(10)):
    print(exc)

## Brightway ❤️ Pandas

In [ ]:
df = bd.Database("US EEIO 1.1").nodes_to_dataframe()
df

In [ ]:
df = bd.Database("US EEIO 1.1").edges_to_dataframe()
df

# Moar columnzzz???

In [ ]:
def remove_target_database(node, edge, row):
    del row['target_database']
    
def food_sector(node, edge, row):
    row['is_food'] = 'food' in edge['name'].lower()

In [ ]:
df = bd.Database("US EEIO 1.1").edges_to_dataframe(formatters=[remove_target_database, food_sector])
df

In [ ]:
exiobase.nodes_to_dataframe()

In [ ]:
exiobase.edges_to_dataframe()

We can also get a dataframe of the edges for a specific node. Here we get all edges, but you can filter this further with the edge constructors `.production()`, `.technosphere()`, and `.biosphere()`.

In [ ]:
df = activity.technosphere().to_dataframe()
df

## I can haz LCA?

In [ ]:
lca = bc.LCA({product: 1}, method=('Impact Potential', 'HRSP'))
lca.lci()
lca.lcia()
lca.to_dataframe()

In [ ]:
lca.to_dataframe(matrix_label='biosphere_matrix')

# bw2calc

## Specify `data_objs` and new functional unit

The new calling convention is therefore functional unit **with the IDs for the nodes** and **datapackages as `data_objs`**. Here is an example:

In [ ]:
database_dp = bd.Database("US EEIO 1.1").datapackage()
lcia_dp = bd.Method(('Impact Potential', 'HC')).datapackage()

In [ ]:
lca = bc.LCA({product.id: 1}, data_objs=[database_dp, lcia_dp])
lca.lci()
lca.lcia()
lca.score

The old calling convention will still work, but only if you have `bw2data` installed, the correct project selected, etc.

## `bw2data.prepare_lca_inputs`

If you don't want to remember the new calling convention, you can use a helper function: `bw2data.prepare_lca_inputs`. It will return three things: A new demand dictionary, the datapackages, and (if `remapping=True`, the default) dictionaries to allow you to map matrix indices back to Brightway (database, code) keys.

In [ ]:
fu, dps, remapping = bd.prepare_lca_inputs({bd.get_node(name='Steel'): 1}, ('IPCC', 'simple'))

In [ ]:
fu

In [ ]:
dps

In [ ]:
remapping

## No automatic remapping

Previously, `bw2calc.LCA` would automatically change the integer values given in `bw2data.mapping` to keys. This is no longer the case, as we assume normal behaviour in the future is to prefer node IDs to keys, and also don't have any guarantees on whether `bw2data` is available. Therefore, **you need to call LCA.remap_inventory_dicts() manually**.

In [ ]:
lca = bc.LCA(demand=fu, data_objs=dps, remapping_dicts=remapping)
lca.lci()
lca.lcia()

In [ ]:
lca.demand

In [ ]:
lca.remap_inventory_dicts()

In [ ]:
lca.demand

## `.redo_lci` ➡️ `.lci`, `.redo_lcia` ➡️ `.lcia`

The preferred way to do additional LCI or LCIA calculations has changed, and has been simplified to the same way we do an initial caluculation, namely with `.lci(new_demand_dict)` and `.lcia(new_demand_dict)`.

In [ ]:
lca = bc.LCA(demand=fu, data_objs=dps)
lca.lci()
lca.lcia()

for act in [
    bd.get_node(name='Driving an combustion car'),
    bd.get_node(name='Driving an electric car'), 
]:
    lca.lcia({act.id: 1})
    print(lca.score, act)

## `LCA` object can now do Monte Carlo

`MonteCarloLCA` ➡️ `LCA(..., use_distributions=True)`

In [ ]:
lca = bc.LCA(
    demand={bd.get_node(name='Driving an electric car'): 1}, 
    method=('IPCC', 'uncertain'),
    use_distributions=True
)
lca.lci()
lca.lcia()

for _ in zip(range(10), lca):
    print(lca.score)

### `keep_first_iteration`

Sometimes you want to keep the values sampled when you set up the LCA object, instead of getting the next value in a sequence when you start iterating over Monte Carlo results. In this case, just call `.keep_first_iteration` and it will skip the first iteration step.

In [ ]:
lca = bc.LCA(
    demand={bd.get_node(name='Driving an electric car'): 1}, 
    method=('IPCC', 'uncertain'),
    use_distributions=True
)
lca.lci()
lca.lcia()
lca.keep_first_iteration()
print("Score after setup:", lca.score)

for _ in zip(range(4), lca):
    print("In loop:", lca.score)

## New `.dicts` accessor

The old methods still work (well, they will in the next dev release :), but the preferred way is to do the following:

In [ ]:
lca = bc.LCA({product: 1})
lca.lci()

To matrix index:

In [ ]:
act_matrix = lca.dicts.activity[activity.id]
act_matrix

To database ID:

In [ ]:
lca.dicts.activity.reversed[act_matrix], activity.id

And the same for products and biosphere flows:

In [ ]:
lca.dicts.product[product.id], lca.dicts.biosphere[emission.id]

# What's coming?

## Easier IO

* `bw2io.useeio()`: In production
* `bw2io.exiobase_monetary()`: In development branch
* `bw2io.ecoinvent()`: Hackathon?

## SimaPro 🤝 ecoinvent

https://github.com/brightway-lca/simapro_ecoinvent_elementary_flows

## Brightway 3

[Brightway 3 strategic plan](https://github.com/brightway-lca/enhancement-proposals/blob/main/Brightway%20strategic%20development%20plan.md)

## What's all this then? 

Cauldron? DdS? Brightway?